In [7]:
import numpy as np
from qiskit import *

id2 = np.identity(2)
ox = np.array([[0,1],[1,0]])
oy = np.array([[0+0j,0-1j],[0+1j,0+0j]])
oz = np.array([[1,0],[0,-1]])

h_gate = 1/np.sqrt(2)*(ox+oz)

In [22]:
# Math Coded

#State Generator Code from Elizabeth

#define the number of qubits, currently we are using 3
qubits = 3
#define the total number of possibilities 
length = 2**qubits
#deine a list to hold the bianary representation
stat = []
#define a function to give that bianary representation from an integer
getbinary = lambda x, n: format(x, 'b').zfill(n)
#loop through the integers to get the numbers in bianary
#aka, the possible states 
for i in range(length):
    stat.append(getbinary(i, qubits))
    
#define a list to hold the vector representation of the states 
state = []
#loop over all states 
for i in range(len(stat)):
    #break up states by each value 
    fornow = list(stat[i])
    #define a list to hold the vector form 
    fornow2 = []
    #loop over string values, if add vector 1, if 0, add vector 0
    for j in range(len(fornow)):
        if fornow[j] == '1':
            fornow2.append([0,1])
        if fornow[j] == '0':
            fornow2.append([1,0])
    #add the list of vectors to the full list with all states 
    state.append(fornow2)
#convert the list of lists of lists into a numpy array 
state = np.array(state)


#vector matrix vector mul
def vmv(vec1,vec2,mat):
    sum = np.matmul(vec2,np.matmul(vec1,mat))
    return sum

def gs(A):
    A = (A)
    A = 1.0*A
    #delete the all zero columns in the original matrix
    A = np.delete(A, np.argwhere(np.all(A[:,...] < 1e-6, axis=0)), axis=0)
    (m,n) = A.shape
    O = np.zeros((m,n), dtype=A.dtype)
    o_i = 0
    ndel = m-1
    for i in range(m):
        #print(O)
        q = A[i] # i-th row of A
        for j in range(o_i):
            q = q - np.dot(np.conj(O[j]), q) * O[j]
        if (np.linalg.norm(q)/np.linalg.norm(A[i]) < 1e-6):
            O = np.delete(O, obj=ndel, axis=0)
            ndel -= 1
        else:
            q = q / np.linalg.norm(q)
            O[o_i] = q
            o_i += 1
    return O


#finding eigenvalues and eigenvectors of a given matrix eg. Hamiltonian generated above. a 2d list, 0iter - 
def eigens(matrix):
    evals,evec = np.linalg.eig(matrix)
    return evals,(evec)

#arrangement of eigenvectors with respect to eigenvalues from smallest to greatest
#evals refers to the eigenvalues, this is a list of same lenght to the evecs, which refers to the eigenvectors. 
#This code sorts the eigenvectors under one condition,
def arr(evals,evec):
    item = np.argsort(evals)
    vectors = []
    for i in range(len(evals)):
        vectors.append(evec[item[i]])
    return vectors


def energy_calculation(ham):
    arrangedevecs = arr(eigens(np.array(ham,float))[0],np.transpose(eigens(np.array(ham,float))[1]))
    projevecs = projector_all@arrangedevecs
    gevecs = gs(projevecs)@swap@cp3
    Energ= []
    for i in range(len(gevecs)):
        vec = gevecs[i]
        Energ.append(np.dot((vec),np.matmul(ham,np.transpose(vec)))/np.dot(vec,vec))  
    return Energ


In [23]:
#Hamiltonian Code

#coefficients of hamiltonian with term variation
def hcd(a,b,mass,photon,interaction):
    st = (1/2)*vmv(a[0],b[0],oz)
    nd = (1/2)*vmv(a[2],b[2],oz)
    rd = (1/2)*vmv(a[1],b[1],ox)
    th = (1/4)*vmv(a[1],b[1],oz)*vmv(a[0],b[0],ox)*vmv(a[2],b[2],ox)
    fth = (1/4)*vmv(a[1],b[1],oz)*vmv(a[0],b[0],oy)*vmv(a[2],b[2],oy)
    coeff = mass*(st - nd) + photon*rd + interaction*(th + fth)
    return coeff

#generating the matrix with term alteration
def Ham(statevectors,mass,photon,interacting):
    zeromatrix = np.zeros((len(statevectors),len(statevectors)))
    for i in range(len(statevectors)):
        vec1 = statevectors[i]
        for j in range(len(statevectors)):
            vec2 = statevectors[j]
            coefficient = hcd(vec1,vec2,mass,photon,interacting)
            zeromatrix[i,j] = coefficient
    return zeromatrix

In [24]:
###########################################################
#Gauge transformation G = Z_0 X_1
def gauge_Z0_X1_coefficients(state_1,state_2):
    coefficient = vmv(state_1[0],state_2[0],oz)*vmv(state_1[1],state_2[1],ox)*vmv(state_1[2],state_2[2],[[1,0],[0,1]])
    return coefficient

#gauge transformation G = Z_0 X_1
gauge_z0_x1 = [[],[],[],[],[],[],[],[]]

for i in range(len(state)):
    vector_1 = state[i]
    for j in range(len(state)):
        vector_2 = state[j]
        coefficient = gauge_Z0_X1_coefficients(vector_1,vector_2)
        gauge_z0_x1[j].append(coefficient)

#projector 1/2(I+G_1)
projector_z0_x1 = np.array(gauge_z0_x1,float)
for i in range(len(state)):
    projector_z0_x1[i][i] =1 + projector_z0_x1[i][i]
projector_z0_x1 = (1/2)*np.array(projector_z0_x1,float)

##############################################################

#Gauge transformation G = - X_1 Z_2 
def gauge_X1_Z2_coefficients(state_1,state_2):
    coefficient = -vmv(state_1[1],state_2[1],ox)*vmv(state_1[2],state_2[2],oz)*vmv(state_1[0],state_2[0],[[1,0],[0,1]])
    return coefficient

#gauge transformation G = - X_1 Z_2 
gauge_x1_z2 = [[],[],[],[],[],[],[],[]]

for i in range(len(state)):
    vector_1 = state[i]
    for j in range(len(state)):
        vector_2 = state[j]
        coefficient = gauge_X1_Z2_coefficients(vector_1,vector_2)
        gauge_x1_z2[j].append(coefficient)    

 #projector  1/2 ( 1 + X_1 Z_2 )
projector_x1_z2 = np.array(gauge_x1_z2,float)
for i in range(len(state)):
    projector_x1_z2[i][i] = 1 + projector_x1_z2[i][i]
projector_x1_z2 = (1/2)*np.array(projector_x1_z2,float)


###########################

projector_all = projector_x1_z2@projector_z0_x1

###########################


# 3 Qubit CP Transformation
cp3 = np.zeros((8,8))    
for i in range(len(state)):
    for j in range(len(state)):
        cp3[i][j] = cp3[i][j] + vmv(state[i][0],state[j][0],ox)*vmv(state[i][2],state[j][2],ox)*vmv(state[i][1],state[j][1],id2)
        
#Swap Matrix for 3 qubit case
swap = np.eye(8)


swap[1,1]= 0 
swap[4,4]= 0 
swap[1,4]= 1 
swap[4,1]= 1 
swap[3,3]= 0
swap[6,6]= 0
swap[3,6]= 1
swap[6,3]= 1

#CP transformation

cp = swap@cp3


#Photon Basis Vectors

#1-0 state
minus10 = np.kron(np.kron(ox@[1,0],h_gate@ox@[1,0]),[1,0])
#print(minus10)

# 0+1 state
plus01 = np.kron(np.kron([1,0],h_gate@[1,0]),ox@[1,0])
#print(plus01)


In [25]:
def state_circuit(mass,interaction,E_n=0):
    hamil = Ham(state,mass,1,interaction)
    eigenvec = eigens(hamil)[1]
    eigenval = eigens(hamil)[0]
    arrangedvecs = arr(eigenval,eigenvec)
    projected = projector_all@arrangedvecs
    gsvecs = gs(np.transpose(projected))

    vec_0 = gsvecs[0]
    vec_1 = gsvecs[1]

    a_0 = np.dot(plus01,vec_0)
    a_1 = np.dot(plus01,vec_1)


    t_0 = 2*np.arccos(a_0)
    t_1 = 2*np.arccos(a_1)
    thetas = [t_0.real,t_1.real]
    
    q = QuantumRegister(3)
    qc=QuantumCircuit(q)
    qc.rx(thetas[E_n],1)
    qc.h(1)
    qc.cnot(1,0)
    qc.cnot(1,2)
    qc.x(2)
    qc.h(1)


    return(qc)

print(state_circuit(1,1))

                         ┌───┐          
q3_0: ───────────────────┤ X ├──────────
      ┌────────────┐┌───┐└─┬─┘     ┌───┐
q3_1: ┤ Rx(5.0605) ├┤ H ├──■────■──┤ H ├
      └────────────┘└───┘     ┌─┴─┐├───┤
q3_2: ────────────────────────┤ X ├┤ X ├
                              └───┘└───┘


/tmp/ipykernel_60/1816116386.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  zeromatrix[i,j] = coefficient


In [41]:
def energy_calculation(ham):
    arrangedevecs = arr(eigens(np.array(ham,float))[0],(eigens(np.array(ham,float))[1]))
    projevecs = projector_all@np.transpose(arrangedevecs)
    gevecs = gs(np.transpose(projevecs))@swap@cp3
    Energ= []
    for i in range(len(gevecs)):
        vec = gevecs[i]
        Energ.append(np.dot(np.transpose(vec),np.matmul(ham,np.transpose(vec)))/np.dot(vec,vec))  
    return Energ

def magic_function(mass,interaction,E_n=0):
    hamil = Ham(state,mass,1,interaction)
    eigenvec = eigens(hamil)[1]
    eigenval = eigens(hamil)[0]
    arrangedvecs = arr(eigenval,eigenvec)
    projected = projector_all@arrangedvecs
    gsvecs = gs(np.transpose(projected))

    vec_0 = gsvecs[0]
    vec_1 = gsvecs[1]

    a_0 = np.dot(plus01,vec_0)
    a_1 = np.dot(plus01,vec_1)


    t_0 = 2*np.arccos(a_0).real
    t_1 = 2*np.arccos(a_1).real

    thetas = [t_0,t_1]
    
    energies = energy_calculation(hamil)
    
    
    return(thetas[E_n],energies[E_n].real)

magic_function(1,1,0)

/tmp/ipykernel_60/1816116386.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  zeromatrix[i,j] = coefficient


(5.060538023965381, 2.5157547559736297)